In [4]:
import pandas as pd
from datetime import datetime, timezone,timedelta,date
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
from itertools import islice
import json
from web3 import Web3
import matplotlib.ticker as mtick
path = ""
path_output=""

In [2]:
inch_address=['0xe4c577bdec9ce0f6c54f2f82aed5b1913b71ae2f', 
            '0x0000000006adbd7c01bc0738cdbfc3932600ad63', 
            '0x0000000053d411becdb4a82d8603edc6d8b8b3bc',
            '0x000005edbbc1f258302add96b5e20d3442e5dd89', 
            '0x0000000f8ef4be2b7aed6724e893c1b674b9682d',
            '0x111112549cfedf7822eb11fbd8fd485d8a10f93f',
            '0x111111254b08ceeee8ad6ca827de9952d2a46781', 
            '0x11111254369792b2ca5d084ab5eea397ca8fa48b', 
            '0x111111125434b319222cdbf8c261674adb56f3ae', 
            '0x11111112542d85b3ef69ae05771c2dccff4faa26', 
            '0x1111111254fb6c44bac0bed2854e76f90643097d', 
            '0x1111111254eeb25477b68fb85ed929f73a960582']

In [3]:
#2022 1inch involved transaction data
# data origin from EigenPhi
df_1inch_allTxs=pd.read_csv(path+'all_1inch_txs.csv.gz',header=None, names
                         =['block','index','tx_hash','from','to','value','transfer_count','input_data','transfers'])

In [4]:
df_1inch_allTxs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714568 entries, 0 to 3714567
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   block           int64 
 1   index           int64 
 2   tx_hash         object
 3   from            object
 4   to              object
 5   value           object
 6   transfer_count  int64 
 7   input_data      object
 8   transfers       object
dtypes: int64(3), object(6)
memory usage: 255.1+ MB


In [5]:
df_to_inch_allTxs=df_1inch_allTxs[df_1inch_allTxs['to'].isin(inch_address)]

In [6]:
# 2022 matcha transactions as victims for sandwich attacks
# data origin from EigenPhi
df_1inch_sandwich_txhash = pd.read_csv(path_output+'1inch_victim_txlist.csv')

df_1inch_sandwich_transfer=df_to_inch_allTxs[df_to_inch_allTxs['tx_hash'].isin(df_1inch_sandwich_txhash['tx_hash'])]


In [5]:
df_1inch_sandwich=pd.read_csv(path_output+'1inch_victim_txs.csv')

In [10]:
df_1inch_sandwich['datetime']=df_1inch_sandwich['blockTimestamp'].apply(lambda x: 
                                                                        datetime.fromtimestamp(x,tz=timezone.utc))
df_1inch_sandwich['datetime'] = pd.to_datetime(df_1inch_sandwich['datetime']).dt.tz_localize(None)

In [ ]:
# calculat the volume that has been attacked

In [11]:
df_1inch_sandwich_price=pd.read_csv(path_output+'1inch_sandwich_withPrice.csv')

In [17]:
# filtered the transfer data from the victim transaction data
df_all_transfer=df_1inch_sandwich_transfer['transfers'].astype('string').fillna('').apply(lambda x: x.split('|')).explode().reset_index()

df_all_transfer[['token','transfer_from','transfer_to','amount']]=df_all_transfer['transfers'].str.split(';', expand=True)

df_all_transfer[['tx_hash','from','to']]=df_to_inch_allTxs.loc[df_all_transfer['index']].reset_index()[['tx_hash','from','to']]

df_victim_transfer=df_all_transfer[df_all_transfer['tx_hash'].isin(df_1inch_sandwich_txhash['tx_hash'])]

In [19]:
df_1inch_sandwich_flat = df_1inch_sandwich['victimTxs'].apply(lambda x: x.rstrip().split(' ')).explode().reset_index()
df_1inch_sandwich_flat[['transactionHash','lps'
                        ]]=df_1inch_sandwich.loc[df_1inch_sandwich_flat['index']].reset_index()[['transactionHash','lps']]

In [21]:
df_merge_1=df_1inch_sandwich_flat[['transactionHash','victimTxs','lps']].merge(
    df_victim_transfer[['tx_hash','transfer_from','transfer_to','token','amount']],left_on='victimTxs',
    right_on='tx_hash',how='left')

In [22]:
df_merge_1['lps']=df_merge_1['lps'].apply(lambda x: x.split(' '))

df_merge_1=df_merge_1.explode('lps')

In [23]:
df_victim_volume=df_merge_1[
    (df_merge_1['lps']==df_merge_1['transfer_from']) | (df_merge_1['lps']==df_merge_1['transfer_to'])]

In [24]:
df_token_decimals=pd.read_csv(path+'token_decimals.csv')
df_token_decimals=df_token_decimals[['address','decimals']].groupby('address').agg({'decimals':'max'}).reset_index()

In [26]:
#calculate the victim volume based on the token price and transfer amount
df_victim_volume=df_victim_volume.merge(
    df_token_decimals[['address','decimals']],left_on='token',right_on='address',how='left')

df_victim_volume=df_victim_volume.merge(df_1inch_sandwich_price,on='transactionHash',how='left')

df_victim_volume=df_victim_volume[df_victim_volume['token_x']==df_victim_volume['token_y']].reset_index()

df_victim_volume['amount']=df_victim_volume['amount'].astype(float)*df_victim_volume['decimals'].apply(lambda x: 10**(-x))

df_victim_volume['price']=df_victim_volume['price'].astype(float)

df_victim_volume['V_volume']=df_victim_volume['price']*df_victim_volume['amount']

df_victim_volume=df_victim_volume.dropna()

def select_value(group):
    if '0xdac17f958d2ee523a2206206994597c13d831ec7' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xdac17f958d2ee523a2206206994597c13d831ec7','V_volume'].values[0]
    elif '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']['V_volume'].values[0]
    elif '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']['V_volume'].values[0]
    elif '0x6b175474e89094c44da98b954eedeac495271d0f' in group['token_y'].values:
        return group.loc[group['token_y'] == '0x6b175474e89094c44da98b954eedeac495271d0f']['V_volume'].values[0] 
    elif '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599' in group['token_y'].values:
        return group.loc[group['token_y'] == '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599']['V_volume'].values[0] 
    else:
        return group['V_volume'].max()

df_victim_volume_final=df_victim_volume.groupby(
    ['transactionHash','victimTxs']).apply(select_value).reset_index()

In [49]:
df_victim_volume_final.to_csv(path+'1inch_victim_volume.csv')